<a href="https://colab.research.google.com/github/anilbhatt1/Deep_Learning_EVA4_Phase1/blob/master/Master_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mounting the drive 
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd gdrive/My Drive/project_folder

/content/gdrive/My Drive/project_folder


In [3]:
!git clone https://github.com/anilbhatt1/Deep_Learning_EVA4_Phase1

Cloning into 'Deep_Learning_EVA4_Phase1'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 445 (delta 24), reused 0 (delta 0), pack-reused 389
Receiving objects: 100% (445/445), 4.14 MiB | 8.97 MiB/s, done.
Resolving deltas: 100% (261/261), done.


In [5]:
%cd Deep_Learning_EVA4_Phase1/S7_CIFAR10_Modular

/content/gdrive/My Drive/project_folder/Deep_Learning_EVA4_Phase1/S7_CIFAR10_Modular


In [6]:
! ls

models	README.md


In [0]:
from torchvision import transforms
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm

In [9]:
from models import S7_Datatransforms,S7_Model,S7_Train_losses,S7_Dataloaders,S7_Test_losses

SyntaxError: ignored

# Get the CIFAR10 data set

In [0]:
from torchvision import datasets 

inpdata = datasets.CIFAR10('./data',train=True,download=True)



Files already downloaded and verified


In [0]:

# add path to our library in default search path
import sys
sys.path.append('/content/drive/My Drive/EVA4')

# See the data
Let us see some training images

# Transformations 

In [0]:
import numpy as np
import S7-datatransforms
from torchvision import transforms

mean = (0.49139968, 0.48215841, 0.44653091)
stdev = (0.24703223, 0.24348513, 0.26158784)

trans = Transforms_customs.Transforms(normalize=True, mean=channel_means, stdev=channel_stdevs)

# Train Phase transformations
train_transforms = trans.train_transforms([
                                       transforms.RandomRotation((-15.0, 15.0), fill=tuple(fillmeans)),
                                       transforms.RandomHorizontalFlip(),
                                       ], 
                                       
                                       )

            
# Test Phase transformations
test_transforms = trans.test_transforms()

SyntaxError: ignored

# Create Train and Test Split

In [0]:
train = datasets.CIFAR10('./data', train=False, download=True, transform=train_transforms)
test = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

NameError: ignored

# Data Loader arguments

In [0]:
import S7-data-loader

dataloader = S7-data-loader.DataLoader()

# train dataloader
train_loader = dataloader.load(train)

# test dataloader
test_loader = dataloader.load(test)

SyntaxError: ignored

# Model 

In [0]:
import S7-Model 
import torch 
!pip install torchsummary
from torchsummary import summary
model = S7-Model.Net()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model.to(device)
summary(model, input_size=(3, 32, 32))



# Train and Test the model 

In [0]:
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
EPOCHS = 20
L2lambda = 0
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True, weight_decay=L2lambda)
#scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.5, steps_per_epoch=len(train_loader), epochs=EPOCHS)
L1lambda = 0
model.gotrain(optimizer, train_loader, test_loader, EPOCHS, "/content/drive/My Drive", scheduler, True, L1lambda)